# Slider Demo 

This is a very simple interactive demo to show how sliders could work.

** NOTE: please wait until this page loads completely**

In [ ]:
%matplotlib inline 
from ipywidgets import *
import burnman
from burnman import minerals
import numpy as np
import matplotlib.pyplot as plt

Let us make a rock with x % perovskite and 1-x % periclase and plot vs PREM:

In [ ]:
@interact(pv=(0.0, 1.0))
def plot(pv):
    rock = burnman.Composite([minerals.SLB_2011.mg_perovskite(),
                              minerals.SLB_2011.periclase()],
                             [pv, 1.0-pv])
    rock.debug_print()
    depths = np.linspace(750e3, 2800e3, 20)
    seismic_model = burnman.seismic.PREM()
    pressure, seis_rho, seis_vp, seis_vs, seis_vphi = seismic_model.evaluate(
        ['pressure', 'density', 'v_p', 'v_s', 'v_phi'], depths)
    temperature = burnman.geotherm.brown_shankland(pressure)
    density, vp, vs, vphi = rock.evaluate(
        ['density', 'v_p', 'v_s', 'v_phi'], pressure, temperature)

    fig = plt.figure(figsize=(9,4))
    plt.subplot(1, 2, 1)    
    plt.plot(pressure / 1.e9, vs / 1.e3, color='b', linestyle='-',
             marker='o', markerfacecolor='b', markersize=4, label='computation')
    plt.plot(pressure / 1.e9, seis_vs / 1.e3, color='k', linestyle='-',
             marker='o', markerfacecolor='k', markersize=4, label='reference')
    plt.title("S wave speed (km/s), %d %% PV" % (pv*100.0))
    plt.xlim(min(pressure) / 1.e9, max(pressure) / 1.e9)
    plt.legend(loc='lower right')
    plt.ylim(5, 8.0)  
    
    plt.subplot(1, 2, 2)    
    plt.plot(pressure / 1.e9, density / 1.e3, color='b',
             linestyle='-', marker='o', markerfacecolor='b', markersize=4)
    plt.plot(pressure / 1.e9, seis_rho / 1.e3, color='k',
             linestyle='-', marker='o', markerfacecolor='k', markersize=4)
    plt.xlim(min(pressure) / 1.e9, max(pressure) / 1.e9)
    plt.xlabel("Pressure (GPa)")
    plt.title("density (kg/m^3)")
    plt.ylim(4.0, 6.0)  


In [7]:
%%html
<script>
// Note:
// This html code block will run the notebook on startup
// and hide all code blocks to provide a clean starting
// experience.

    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells...");
                jupyter.actions.call('jupyter-notebook:run-all-cells');
                //jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
code_show=false; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
function init() { $('div.input').hide();}
$( document ).ready(init);
$( document ).load(init);

</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>